In [ ]:
def hyperopt_objective(params):
    !vw -d data/habr_train.vw \
    -l {params['l']} \
    --quiet
    
    !vw -i habr_model.vw -t \
    -d data/habr_valid.vw \
    -p habr_valid_pred.txt \
    --quiet
    
    with open('habr_valid_pred.txt') as pred_file:
        valid_prediction = [float(popularity) for popularity in pred_file.readlines()]
    score = mean_absolute_error(valid_vals, valid_prediction)
    return score


params_space = {
    'l': hp.uniform('l', 0.2, 0.8), # default = 0.5
}


%%time
trials = Trials()

best_params = fmin(
    hyperopt_objective,
    space=params_space,
    algo=tpe.suggest,
    max_evals=200,
    trials=trials
)

*Как завести vw-hyperopt.py*
1. Установите нужные либы
!pip install hyperopt
!pip install networkx==1.11 - откат одной из связанных библиотек для работы скрипта

2. Найдите, куда установилось
!find / -name “vowpalwabbit” 2>/dev/null
!find / -name “vw-hyperopt.py” 2>/dev/null

3. Подготовьте vw-hyperopt.py
Необходимо добавить encoding=“utf-8” во все функции open() для правильной работы с utf-8 файлами

*Ну и теперь можно удачно запустить.*
Пример строки запуска(параметры изменил)
%run /vowpal_wabbit/utl/vw-hyperopt.py \ --train ../../data/vowpal/habr_traink.vw \ --holdout ../../data/vowpal/habr_tholdout.vw \ --max_evals 200 \ --outer_loss_function squared \ --vw_space ‘--algorithms=sgd -l=0.01..1~L --passes=1..10~I --loss_function=squared ’ --plot

In [11]:
import numpy as np
import pandas as pd
import json
import os
from tqdm import tqdm_notebook
from sklearn.metrics import mean_absolute_error
from datetime import datetime
%matplotlib inline

In [2]:
PATH_TO_DATA = '../../data/arktur_medium/'
!PATH_TO_DATA=../../data/arktur_medium/

In [5]:
!head -1 $PATH_TO_DATA/train.json > $PATH_TO_DATA/train1.json

In [6]:
with open(os.path.join(PATH_TO_DATA, "train1.json")) as inp_json:
    first_json = json.load(inp_json)

In [7]:
first_json.keys()

dict_keys(['domain', '_timestamp', 'published', 'tags', 'url', 'content', 'quality', 'meta_tags', 'image_url', 'author', 'link_tags', 'title', '_spider', '_id'])

In [476]:
first_json["domain"]

'medium.com'

In [152]:
# remove html tags

from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ' '.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [3]:
INP_TRAIN = "train.json"
INP_TEST  = "test.json"
INP_TRAIN_PARSED = "train_parsed.csv"
INP_TEST_PARSED = "test_parsed.csv"
!INP_TRAIN=train.json
!INP_TEST=test.json

OUT_TRAIN = "train.vw"
OUT_PART_TRAIN = "part_train.vw"
OUT_PART_VALID = "part_valid.vw"
OUT_TEST = "test.vw"
!OUT_TRAIN=train.vw
!OUT_PART_TRAIN=part_train.vw
!OUT_PART_VALID=part_valid.vw
!OUT_TEST=test.vw

LOG = True

In [4]:
train_target = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                        'train_log1p_recommends.csv'), 
                           index_col='id')

In [ ]:
# transform data to vw format
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

lmtzr = WordNetLemmatizer()
stop = set(stopwords.words("english") + list(string.punctuation) + ["’", "“", "”"])
tokenize = lambda s: " ".join(lmtzr.lemmatize(i) for i in word_tokenize(s.lower()) if i not in stop)


def to_vw(data, label):
    exclude = set(string.punctuation)
       
    content = strip_tags(data["content"]).replace("\n", " ").replace(":", "").replace("|", "")
    content = tokenize(content)
    
    title = strip_tags(data["title"]).replace("\n", " ").replace(":", "").replace("|", "")
    title = tokenize(title)
    
    author = strip_tags(data["meta_tags"]["author"]).replace("\n", " ").replace(":", "").replace("|", "")
    author = "".join(ch for ch in author if ch not in exclude).replace(" ", "_").lower()
    
    domain = data["domain"].replace("\n", " ").replace(":", "").replace("|", "").lower()
    
    # date time
    dt = datetime.strptime(data["published"]["$date"][:-1], "%Y-%m-%dT%H:%M:%S.%f")
    month = dt.month
    weekday = dt.weekday()
    hour = dt.hour 
    read_time = data["meta_tags"]["twitter:data1"].split(" ")[0]
    read_time = read_time if read_time else 0.1
    
    # binary
    is_weekend = 1 if weekday in (5, 6) else 0
    is_night = 1 if hour in range(0, 7) else 0
    is_morning = 1 if hour in range(7, 12) else 0
    is_noon = 1 if hour in range(12, 19) else 0
    is_eve = 1 if hour in range(19, 24) else 0
    is_image = 1 if data["image_url"] else 0


    out = [
        str(np.log(label)),
        "a_content %s" % content,
        "b_author %s" % author,
        "c_title %s" % title,
        "d_domain %s" % domain,
        "e_num 10:%s 11:%s 12:%s 13:%s" % (month, weekday + 1, hour + 1, read_time),
        "f_binary 0:%s 1:%s 2:%s 3:%s 4:%s 5:%s" % 
        (is_weekend, is_night, is_morning, is_noon, is_eve, is_image)
    ]
    
    return str(" |".join(out).encode('ascii', 'ignore').strip())[2:-1] + "\n"


In [153]:
import string
exclude = set(string.punctuation)

def to_vw1(label, content, tags, data):
    author = strip_tags(data["meta_tags"]["author"]).replace("\n", " ").replace(":", "").replace("|", "")
    author = "".join(ch for ch in author if ch not in exclude).replace(" ", "_").lower()
    
    domain = data["domain"].replace("\n", " ").replace(":", "").replace("|", "").lower()
    
    # date time
    dt = datetime.strptime(data["published"]["$date"][:-1], "%Y-%m-%dT%H:%M:%S.%f")
    month = dt.month
    weekday = dt.weekday()
    hour = dt.hour 
    read_time = data["meta_tags"]["twitter:data1"].split(" ")[0]
    read_time = read_time if read_time else 0.1
    
    # binary
    is_weekend = 1 if weekday in (5, 6) else 0
    is_night = 1 if hour in range(0, 7) else 0
    is_morning = 1 if hour in range(7, 12) else 0
    is_noon = 1 if hour in range(12, 19) else 0
    is_eve = 1 if hour in range(19, 24) else 0
    is_image = 1 if data["image_url"] else 0
    
    out = [
        str(np.log(label)) if LOG else str(label),
        "a_content %s" % content,
        "b_tags %s" % (tags if tags else "none",),
        "d_domain %s" % domain,
        "e_num 10:%s 11:%s 12:%s 13:%s" % (month, weekday + 1, hour + 1, read_time),
        "f_binary 0:%s 1:%s 2:%s 3:%s 4:%s 5:%s" % 
        (is_weekend, is_night, is_morning, is_noon, is_eve, is_image),
        "g_author %s" % author,
    ]
    
    return " |".join(out) + "\n"

In [67]:
def to_csv(label, data):
    # date time
    dt = datetime.strptime(data["published"]["$date"][:-1], "%Y-%m-%dT%H:%M:%S.%f")
#     publish_ts = pd.to_datetime(data['published']['$date'].replace('T', ' '))
    
    read_time = data["meta_tags"]["twitter:data1"].split(" ")[0]
    read_time = read_time if read_time else "0"
        
    out = [
        data["published"]["$date"][:-1],
        str(label),
    ]
    
    return ",".join(out) + "\n"

In [47]:
import codecs

with codecs.open(os.path.join(PATH_TO_DATA, INP_TRAIN), encoding="utf-8") as inp_json, \
     codecs.open(os.path.join(PATH_TO_DATA, INP_TRAIN_PARSED), encoding="utf-8") as inp_parsed, \
     codecs.open(os.path.join(PATH_TO_DATA, "train.csv"), 'w', encoding="utf-8") as out_train:
    N = 52699
#     N = 100        
    header = False
    
    for n, (json_, parsed) in enumerate(tqdm_notebook(zip(inp_json, inp_parsed), total=N)):
        if n == N: break
#         if not header:
#             out_train.write("Time,Label\n")
#             header = True

        json_data = json.loads(json_)
        url, content, tags = parsed.split(",")
        tags = tags.strip()
        
        label = train_target.iloc[n]["log_recommends"]
#         out = to_vw(json_data, label)
        out = to_vw1(label, content, tags, json_data)
#         out = to_csv(label, json_data)
        out_train.write(out)


In [69]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
init_notebook_mode(connected = True)

def plotly_df(df, title = ''):
    data = []

    for column in df.columns:
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',
            name = column
        )
        data.append(trace)

    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link=False)

df = pd.read_csv(os.path.join(PATH_TO_DATA, "test.csv"), parse_dates=['Time'])
# df = df.sort_index()

In [70]:
df.head()

,Time,Label
0,2017-06-23 08:19:34.211,1
1,2017-06-23 14:27:27.083,1
2,2017-06-23 12:36:34.000,1
3,2017-06-23 15:11:42.425,1
4,2017-06-23 15:21:54.977,1


In [38]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

height has been deprecated.



In [71]:
df.groupby([df.Time.dt.year, df.Time.dt.month]).agg({"count"})

Time Label
          count count
Time Time            
2017 1     5046  5046
     2     5025  5025
     3     6103  6103
     4     6604  6604
     5     8577  8577
     6     8137  8137

In [21]:
df1 = df.tail(1000)

In [22]:
plotly_df(df1, title = "Online users")

In [155]:
# split train into part_train and part_valid

!split -l 36889 $PATH_TO_DATA/$OUT_TRAIN $PATH_TO_DATA/$OUT_TRAIN"_"

!mv $PATH_TO_DATA/$OUT_TRAIN"_aa" $PATH_TO_DATA/$OUT_PART_TRAIN
!mv $PATH_TO_DATA/$OUT_TRAIN"_ab" $PATH_TO_DATA/$OUT_PART_VALID

In [156]:
# from validation target file 

!cut -f 1 -d ' ' $PATH_TO_DATA/$OUT_PART_VALID > $PATH_TO_DATA/part_valid_target.txt

In [ ]:
%%time

# !rm $PATH_TO_DATA/medium.cache 
!vw -d $PATH_TO_DATA/$OUT_PART_TRAIN -k --cache_file=$PATH_TO_DATA/medium.cache \
  --loss_function squared --ngram=a2 --passes 5 -b 26 -l 1 --power_t=0.5 --l2=1e-5 \
  -f $PATH_TO_DATA/model.vw #--readable_model=$PATH_TO_DATA/readable_model.vw

Generating 2-grams for a namespaces.
using l2 regularization = 1e-05
final_regressor = ../../data/arktur_medium//model.vw
Num weight bits = 26
learning rate = 1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = ../../data/arktur_medium//medium.cache
Reading datafile = ../../data/arktur_medium//part_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
3.735672 3.735672            1            1.0   1.9328   0.0000     1178
2.180715 0.625757            2            2.0   1.7891   0.9980      849
1.720227 1.259739            4            4.0   0.8340   0.1097      405
1.073640 0.427052            8            8.0   1.6958   1.9328     4483
1.009323 0.945007           16           16.0   0.3266   0.3884      705
0.692505 0.375686           32           32.0   1.3425   1.7541     1499
0.541030 0.389556           64           64.0   0.0940   0.1249    

In [160]:
# predict for validation and MAE

!vw -i $PATH_TO_DATA/model.vw -t -d $PATH_TO_DATA/$OUT_PART_VALID \
-p $PATH_TO_DATA/part_valid_predictions.txt --quiet

from sklearn.metrics import mean_absolute_error

y_true = np.loadtxt(PATH_TO_DATA + '/part_valid_target.txt')
y_pred = np.loadtxt(PATH_TO_DATA + '/part_valid_predictions.txt')

mean_absolute_error(y_true, y_pred)

0.50900557937563662

In [80]:
from sklearn.metrics import mean_absolute_error

y_true = np.loadtxt(PATH_TO_DATA + '/test_target.txt')
for filename in os.listdir(PATH_TO_DATA + "/submit"):
    if filename.startswith("etha"): continue
    y_target = pd.read_csv(PATH_TO_DATA + '/submit/' + filename)
    y_pred = y_target["log_recommends"]

    print(mean_absolute_error(y_true, y_pred), filename)

1.15308976943 squared_b_20_l_0.5_t_0.5.csv
1.15625028783 at_squared_b_21_l_0.5_t_0.5.csv
1.14395825558 log_na2_p5_b26_l_1_p05_l2_1e5.csv
1.12767829627 log_abef_n2_p5_b24_l05_p05_l2_1e5.csv
1.16862095356 squared_b_21_l_0.5_t_0.5_l2_1e-5.csv
1.1935572218 na2_p5_b26_l_1_p05_l2_1e5.csv
1.14225266476 squared_b_28_l_0.43_t_0.17.csv


| Arktur | File| Features | VW Params | MAE |
|------|------|------|------|------|
| 0.89639 | squared_b_20_l_0.5_t_0.5.csv | content | --loss_function squared --ngram 2 --passes 3 -b 20 -l 0.5 --power_t 0.5 | |
| 0.88511 | at_squared_b_21_l_0.5_t_0.5.csv | no punctuation, author, title, content | --loss_function squared --ngram 2 --passes 3 -b 21 -l 0.5 --power_t 0.5 | 1.047 |
| not| not | no pun, author, title, content, month, dayofweek, hour | --loss_function squared --ngram 2 --passes 3 -b 21 -l 0.5 --power_t 0.5 | 1.027 |
| not| not | no pun-scale, author, title, content, month, dayofweek, hour | --loss_function squared --ngram 2 --passes 3 -b 21 -l 0.5 --power_t 0.5 | 1.030 |
| 0.88297 | squared_b_21_l_0.5_t_0.5_l2_1e-5.csv | nonpun, author, title, content, month, dayofweek, hour | --loss_function squared --ngram=2 --passes 3 -b 21 -l 0.5 --power_t 0.5 --l2=1e-5 | 1.019 |
| 0.90253 | squared_b_21_l_0.7_t_0.5_l2_1e-5_new_features.csv | nonpun, author, title, content, month, dayofweek, hour, bin, domain | --loss_function squared --ngram=2 --passes 3 -b 21 -l 0.7 --power_t 0.5 --l2=1e-5 | 1.016 |
| 0.89520 | squared_b_21_l_0.5_t_0.5_new_features.csv | nonpun, author, title, content, month, dayofweek, hour, bin, domain | --loss_function squared --ngram=2 --passes 3 -b 21 -l 0.5 --power_t 0.5 | 1.026 |
| 0.85777 | g_3_p_5_b_21_l_0.5_t_0.5_l1_lower.csv | nonpun, low, content, author, title, domain, month, dayofweek, hour, bin | --loss_function squared --ngram=3 --passes 5 -b 21 -l 0.5 --power_t 0.5 --l2=1e-5 | 1.014 |
| 0.85671 | g_3_p_5_b_21_l_0.5_t_0.5_l1_lower.csv | nonpun, low, content, author, title, domain, month, dayofweek, hour| --loss_function squared --ngram=3 --passes 5 -b 21 -l 0.5 --power_t 0.5 --l2=1e-5 | 1.014 |
| 0.85415 | filtered_content_n2_p5_b28_l0.5_p0.5_l21e-5.csv | f_content | --loss_function squared --ngram=2 --passes 5 -b 28 -l 0.5 --power_t=0.5 --l2=1e-5 | 0.984 |
| 0.82853 | lema_content_n2_p5_b24_l0.5_p0.5_l21e-5.csv | l_content | --loss_function squared --ngram=2 --passes 5 -b 24 -l 0.5 --power_t=0.5 --l2=1e-5 | 0.9813 |
| 0.69766 | log_lema_features_n2_p5_b24_l0.5_p0.5_l21e-5.csv | log_lema_features | --loss_function squared --ngram=2 --passes 5 -b 24 -l 0.5 --power_t=0.5 --l2=1e-5 | log 0.4161 |

In [148]:
# !rm $PATH_TO_DATA/medium.cache 
!vw -d $PATH_TO_DATA/$OUT_TRAIN -k --cache_file $PATH_TO_DATA/medium.cache \
  --loss_function squared --ngram=2 --passes 5 -b 24 -l 0.5 --power_t=0.5 --l2=1e-5 \
  -f $PATH_TO_DATA/model.vw #--readable_model=$PATH_TO_DATA/readable_model.vw

Generating 2-grams for all namespaces.
using l2 regularization = 1e-05
final_regressor = ../../data/arktur_medium//model.vw
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = ../../data/arktur_medium//medium.cache
Reading datafile = ../../data/arktur_medium//train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
3.735672 3.735672            1            1.0   1.9328   0.0000     1181
2.181843 0.628015            2            2.0   1.7891   0.9966      851
1.717136 1.252428            4            4.0   0.8340   0.1174      409
1.067019 0.416902            8            8.0   1.6958   1.9328     4489
0.992346 0.917673           16           16.0   0.3266   0.3991      709
0.680501 0.368656           32           32.0   1.3425   1.7396     1505
0.534910 0.389320           64           64.0   0.0940   0.1465     

In [68]:
# create test data

import codecs

with codecs.open(os.path.join(PATH_TO_DATA, INP_TEST), encoding="utf-8") as inp_json, \
     codecs.open(os.path.join(PATH_TO_DATA, INP_TEST_PARSED), encoding="utf-8") as inp_parsed, \
     codecs.open(os.path.join(PATH_TO_DATA, "test.csv"), 'w', encoding="utf-8") as out_test:
    N = 39492        
    header = False
    
    for n, (json_, parsed) in enumerate(tqdm_notebook(zip(inp_json, inp_parsed), total=N)):
        if n == N: break
        if not header:
            out_test.write("Time,Label\n")
            header = True

        json_data = json.loads(json_)
        url, content, tags = parsed.split(",")
        tags = tags.strip()
        
#         out = to_vw1(1, content, tags, json_data)
        out = to_csv(1, json_data)
        out_test.write(out)

In [150]:
# predict for test

!vw -i $PATH_TO_DATA/model.vw -t -d $PATH_TO_DATA/test.vw \
-p $PATH_TO_DATA/test_predictions.txt --quiet

In [151]:
# create submission

submission = pd.read_csv(PATH_TO_DATA + '/sample_submission.csv', index_col='id')
if LOG:
    submission['log_recommends'] = np.exp(np.loadtxt(PATH_TO_DATA + '/test_predictions.txt'))
else:
    submission['log_recommends'] = np.loadtxt(PATH_TO_DATA + '/test_predictions.txt')
submission.to_csv(PATH_TO_DATA + '/submit/log_abdefg_n2_p5_b24_l05_p05_l2_1e5.csv')

=========================================

In [465]:
%run /vowpal_wabbit/utl/vw-hyperopt.py \
  --train $PATH_TO_DATA/$OUT_PART_TRAIN \
  --holdout $PATH_TO_DATA/$OUT_PART_VALID \
  --max_evals 200 \
  --outer_loss_function squared \
  --vw_space '--algorithms=sgd --ngram=2 --bit_precision=25 --l2=1e-8..1e-1~LO --l1=1e-8..1e-1~LO -l=0.01..10~L --power_t=0.01..1 --passes=1..10~I --loss_function=squared' --plot

2017-11-01 11:26:12,443 INFO     [root/vw-hyperopt:250]: loading true holdout class labels...
2017-11-01 11:26:12,604 INFO     [root/vw-hyperopt:257]: holdout length: 15810
2017-11-01 11:26:12,606 DEBUG    [root/vw-hyperopt:320]: starting hypersearch...
2017-11-01 11:26:12,623 INFO     [hyperopt.tpe/tpe:814]: tpe_transform took 0.004462 seconds
2017-11-01 11:26:12,627 INFO     [hyperopt.tpe/tpe:844]: TPE using 0 trials
2017-11-01 11:26:12,632 INFO     [root/vw-hyperopt:293]: 

Starting trial no.1
2017-11-01 11:26:12,641 INFO     [root/vw-hyperopt:231]: executing the following command (training): vw -d ../../data/arktur_medium//part_train.vw -f ./current.model --holdout_off -c  -l 2.293110126098825 --l2 4.394402312218596e-07 --l1 1.0884538019468025e-06 --ngram 2 --passes 4 --loss_function squared --power_t 0.15312831696127777 --bit_precision 25 
2017-11-01 11:27:06,630 INFO     [root/vw-hyperopt:236]: executing the following command (validation): vw -t -d ../../data/arktur_medium//part_

2017-11-01 11:36:07,833 INFO     [root/vw-hyperopt:283]: parameter suffix: -l 1.294758659621406 --l2 0.00194707552590083 --l1 1.3980930204923765e-08 --ngram 2 --passes 4 --loss_function squared --power_t 0.9794242693622698 --bit_precision 25 
2017-11-01 11:36:07,836 INFO     [root/vw-hyperopt:284]: loss value: 0.244926
2017-11-01 11:36:07,838 INFO     [root/vw-hyperopt:301]: evaluation time for this step: 0:01:06.571576
2017-11-01 11:36:07,964 INFO     [hyperopt.tpe/tpe:814]: tpe_transform took 0.005542 seconds
2017-11-01 11:36:07,966 INFO     [hyperopt.tpe/tpe:842]: TPE using 7/7 trials with best loss 0.244926
2017-11-01 11:36:07,970 INFO     [root/vw-hyperopt:293]: 

Starting trial no.8
2017-11-01 11:36:07,972 INFO     [root/vw-hyperopt:231]: executing the following command (training): vw -d ../../data/arktur_medium//part_train.vw -f ./current.model --holdout_off -c  -l 0.7353818623871349 --l1 4.535829279609697e-06 --ngram 2 --passes 2 --loss_function squared --power_t 0.020465817756

2017-11-01 11:44:30,583 INFO     [root/vw-hyperopt:236]: executing the following command (validation): vw -t -d ../../data/arktur_medium//part_valid.vw -i ./current.model -p ./holdout.pred --holdout_off -c
2017-11-01 11:44:31,772 INFO     [root/vw-hyperopt:283]: parameter suffix: -l 0.03488895272181752 --ngram 2 --passes 2 --loss_function squared --power_t 0.13995598650314572 --bit_precision 25 
2017-11-01 11:44:31,787 INFO     [root/vw-hyperopt:284]: loss value: 0.269393
2017-11-01 11:44:31,789 INFO     [root/vw-hyperopt:301]: evaluation time for this step: 0:00:43.426290
2017-11-01 11:44:31,966 INFO     [hyperopt.tpe/tpe:814]: tpe_transform took 0.010433 seconds
2017-11-01 11:44:31,970 INFO     [hyperopt.tpe/tpe:842]: TPE using 14/14 trials with best loss 0.242033
2017-11-01 11:44:32,007 INFO     [root/vw-hyperopt:293]: 

Starting trial no.15
2017-11-01 11:44:32,013 INFO     [root/vw-hyperopt:231]: executing the following command (training): vw -d ../../data/arktur_medium//part_train

2017-11-01 11:58:03,128 INFO     [root/vw-hyperopt:236]: executing the following command (validation): vw -t -d ../../data/arktur_medium//part_valid.vw -i ./current.model -p ./holdout.pred --holdout_off -c
2017-11-01 11:58:03,565 INFO     [root/vw-hyperopt:283]: parameter suffix: -l 0.1258544581289467 --l2 0.0021794519770999554 --l1 0.0003054781983812668 --ngram 2 --passes 10 --loss_function squared --power_t 0.7609731161979919 --bit_precision 25 
2017-11-01 11:58:03,567 INFO     [root/vw-hyperopt:284]: loss value: 0.271779
2017-11-01 11:58:03,570 INFO     [root/vw-hyperopt:301]: evaluation time for this step: 0:02:15.552367
2017-11-01 11:58:03,682 INFO     [hyperopt.tpe/tpe:814]: tpe_transform took 0.006547 seconds
2017-11-01 11:58:03,684 INFO     [hyperopt.tpe/tpe:842]: TPE using 21/21 trials with best loss 0.222078
2017-11-01 11:58:03,705 INFO     [root/vw-hyperopt:293]: 

Starting trial no.22
2017-11-01 11:58:03,708 INFO     [root/vw-hyperopt:231]: executing the following command (

2017-11-01 12:09:52,935 INFO     [root/vw-hyperopt:293]: 

Starting trial no.28
2017-11-01 12:09:52,937 INFO     [root/vw-hyperopt:231]: executing the following command (training): vw -d ../../data/arktur_medium//part_train.vw -f ./current.model --holdout_off -c  -l 0.19894874997088893 --l2 0.009018642389801285 --l1 0.0003914241279528883 --ngram 2 --passes 9 --loss_function squared --power_t 0.8698646197922101 --bit_precision 25 
2017-11-01 12:11:42,465 INFO     [root/vw-hyperopt:236]: executing the following command (validation): vw -t -d ../../data/arktur_medium//part_valid.vw -i ./current.model -p ./holdout.pred --holdout_off -c
2017-11-01 12:11:42,822 INFO     [root/vw-hyperopt:283]: parameter suffix: -l 0.19894874997088893 --l2 0.009018642389801285 --l1 0.0003914241279528883 --ngram 2 --passes 9 --loss_function squared --power_t 0.8698646197922101 --bit_precision 25 
2017-11-01 12:11:42,824 INFO     [root/vw-hyperopt:284]: loss value: 0.319035
2017-11-01 12:11:42,828 INFO     [roo

...

2017-11-01 13:00:50,731 INFO     [hyperopt.tpe/tpe:814]: tpe_transform took 0.009509 seconds
2017-11-01 13:00:50,733 INFO     [hyperopt.tpe/tpe:842]: TPE using 61/61 trials with best loss 0.214948
2017-11-01 13:00:50,753 INFO     [root/vw-hyperopt:293]: 

Starting trial no.62
2017-11-01 13:00:50,754 INFO     [root/vw-hyperopt:231]: executing the following command (training): vw -d ../../data/arktur_medium//part_train.vw -f ./current.model --holdout_off -c  -l 7.046651625533099 --l2 0.00041436680340823533 --l1 0.00011049375780983791 --ngram 2 --passes 5 --loss_function squared --power_t 0.36800793335420817 --bit_precision 25 
2017-11-01 13:01:49,677 INFO     [root/vw-hyperopt:236]: executing the following command (validation): vw -t -d ../../data/arktur_medium//part_valid.vw -i ./current.model -p ./holdout.pred --holdout_off -c
2017-11-01 13:01:50,106 INFO     [root/vw-hyperopt:283]: parameter suffix: -l 7.046651625533099 --l2 0.00041436680340823533 --l1 0.00011049375780983791 --ngram 2

2017-11-01 13:10:52,519 INFO     [root/vw-hyperopt:284]: loss value: 0.225654
2017-11-01 13:10:52,523 INFO     [root/vw-hyperopt:301]: evaluation time for this step: 0:02:20.203779
2017-11-01 13:10:52,689 INFO     [hyperopt.tpe/tpe:814]: tpe_transform took 0.012958 seconds
2017-11-01 13:10:52,691 INFO     [hyperopt.tpe/tpe:842]: TPE using 68/68 trials with best loss 0.214948
2017-11-01 13:10:52,713 INFO     [root/vw-hyperopt:293]: 

Starting trial no.69
2017-11-01 13:10:52,714 INFO     [root/vw-hyperopt:231]: executing the following command (training): vw -d ../../data/arktur_medium//part_train.vw -f ./current.model --holdout_off -c  -l 0.23433510113427503 --l2 0.008445302169332765 --ngram 2 --passes 9 --loss_function squared --power_t 0.6095679241648939 --bit_precision 25 
2017-11-01 13:12:50,729 INFO     [root/vw-hyperopt:236]: executing the following command (validation): vw -t -d ../../data/arktur_medium//part_valid.vw -i ./current.model -p ./holdout.pred --holdout_off -c
2017-11-0

2017-11-01 13:24:32,646 INFO     [root/vw-hyperopt:283]: parameter suffix: -l 0.14416322084735944 --l2 0.013076657052956528 --ngram 2 --passes 8 --loss_function squared --power_t 0.5673548201328373 --bit_precision 25 
2017-11-01 13:24:32,648 INFO     [root/vw-hyperopt:284]: loss value: 0.211586
2017-11-01 13:24:32,655 INFO     [root/vw-hyperopt:301]: evaluation time for this step: 0:01:50.172443
2017-11-01 13:24:32,767 INFO     [hyperopt.tpe/tpe:814]: tpe_transform took 0.005558 seconds
2017-11-01 13:24:32,769 INFO     [hyperopt.tpe/tpe:842]: TPE using 75/75 trials with best loss 0.211586
2017-11-01 13:24:32,794 INFO     [root/vw-hyperopt:293]: 

Starting trial no.76
2017-11-01 13:24:32,796 INFO     [root/vw-hyperopt:231]: executing the following command (training): vw -d ../../data/arktur_medium//part_train.vw -f ./current.model --holdout_off -c  -l 0.14951904463800159 --l2 0.013844479188561843 --ngram 2 --passes 8 --loss_function squared --power_t 0.6859903067373448 --bit_precision 2

2017-11-01 13:37:32,956 INFO     [root/vw-hyperopt:236]: executing the following command (validation): vw -t -d ../../data/arktur_medium//part_valid.vw -i ./current.model -p ./holdout.pred --holdout_off -c
2017-11-01 13:37:33,620 INFO     [root/vw-hyperopt:283]: parameter suffix: -l 0.1723072922630768 --l2 0.0001911226306365608 --ngram 2 --passes 9 --loss_function squared --power_t 0.9031101877408576 --bit_precision 25 
2017-11-01 13:37:33,624 INFO     [root/vw-hyperopt:284]: loss value: 0.257816
2017-11-01 13:37:33,629 INFO     [root/vw-hyperopt:301]: evaluation time for this step: 0:01:59.774089
2017-11-01 13:37:33,755 INFO     [hyperopt.tpe/tpe:814]: tpe_transform took 0.006001 seconds
2017-11-01 13:37:33,757 INFO     [hyperopt.tpe/tpe:842]: TPE using 82/82 trials with best loss 0.211586
2017-11-01 13:37:33,781 INFO     [root/vw-hyperopt:293]: 

Starting trial no.83
2017-11-01 13:37:33,783 INFO     [root/vw-hyperopt:231]: executing the following command (training): vw -d ../../data/

KeyboardInterrupt: 

In [438]:
!vw -i $PATH_TO_DATA/model.vw -t --invert_hash $PATH_TO_DATA/invert_hash.vw


Generating 2-grams for all namespaces.
ignoring namespaces beginning with: a 
only testing
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
^C
